In [1]:
#!pip install -q openai gradio anthropic python-dotenv

#### For token generation

- OpenAI: https://platform.openai.com/api-keys  
- Anthropic: https://console.anthropic.com/settings/keys  
- Google: https://ai.google.dev/gemini-api
- Huggingface : https://huggingface.co/settings/tokens

In [2]:
from openai import OpenAI
import gradio as gr
from anthropic import Anthropic

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.environ.get('OPENAI_API_KEY','')
anthropic_api_key = os.environ.get('ANTHROPIC_API_KEY','')
google_api_key = os.environ.get('GOOGLE_API_KEY','')

In [4]:
def openai_api_stream_chat(openai_instance,model,history):
    stream = openai_instance.chat.completions.create(model=model, messages=history, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [5]:
openai = OpenAI(api_key=openai_api_key)
def openai_stream_chat(model,history):
    return openai_api_stream_chat(openai,model,history)

In [6]:
gemini_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
def gemini_stream_chat(model,history):
    return openai_api_stream_chat(gemini_client,model,history)

In [7]:
anthropic = Anthropic(api_key=anthropic_api_key)
def claude_stream_chat(model, history,system_message):

    # Create streaming response
    stream = anthropic.messages.create(
        model=model,
        messages=history,
        stream=True,
        max_tokens=4096,
        system = system_message
    )

    response = ""
    for chunk in stream:
        if chunk.type == "content_block_delta":
            response += chunk.delta.text or ''
            yield response

In [8]:
import re
def is_not_blank(string):
    return bool(re.search(r'\S', string))

In [9]:
model_choices = ["claude-3-sonnet-20240229", "gpt-4o-mini","gpt-4o", "gpt-3.5-turbo","gemini-1.5-flash"]
css = """
.custom-row {
    display: flex;
    align-items: center;
}
footer {display: none !important}
.gradio-container {min-height: 0px !important}
"""
def call_llm(history, selected_model: str,system_message):
    print(system_message)
    cleaned_history = []
    if system_message and is_not_blank(system_message) and (not 'claude' in selected_model):
        cleaned_history.append({"role": 'system', "content": system_message})
    for msg in history:
        cleaned_history.append({"role": msg["role"], "content": msg["content"]})
    if 'gpt' in selected_model:
        return openai_stream_chat(selected_model, history=cleaned_history)
    elif 'claude' in selected_model:
        return claude_stream_chat(selected_model, history=cleaned_history,system_message=system_message)
    elif 'gemini' in selected_model:
        return gemini_stream_chat(selected_model, history=cleaned_history)

with gr.Blocks(css=css,title= 'LLM chatbots') as ui:
    selected_model = gr.State(value=model_choices[0])  # Use gr.State to store the current model

    dropdown = gr.Dropdown(
        label="Choose a model",
        choices=model_choices,
        interactive=True,
        value=model_choices[0]
    )
    
    system_message = gr.Textbox(label="System message:")

    chatbot = gr.Chatbot(height=500, type="messages")
    entry = gr.Textbox(label="Chat with our AI Assistant:")
    clear = gr.Button("Clear")

    def update_model(new_model):
        return new_model

    def do_entry(message, history, current_model,system_message_text):
        history += [{"role": "user", "content": message}]
        yield "", history
        for chunk in call_llm(history, current_model,system_message_text):
            yield '', [*history, {"role": "assistant", "content": chunk}]

    # Update the model state when dropdown changes
    dropdown.change(
        update_model,
        inputs=[dropdown],
        outputs=[selected_model]
    )

    # Pass the current model state to do_entry
    entry.submit(
        do_entry,
        inputs=[entry, chatbot, selected_model,system_message],
        outputs=[entry, chatbot]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True,share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://4c1502c95175d2aa8c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


all query will in the context of flutter app development with dart
all query will in the context of flutter app development with dart
all query will in the context of flutter app development with dart
all query will in the context of flutter app development with dart
all query will in the context of flutter app development with dart
all query will in the context of flutter app development with dart
